In [1]:
#import findspark
import findspark
findspark.init()

In [2]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark import SparkConf
import re
import pyspark.sql.functions as func
#from pyspark.ml.feature import Tokenizer, HashingTF, IDF 
#from pyspark.ml.classification import LogisticRegression
#from pyspark.ml import Pipeline
from pyspark.sql.functions import col


from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer, StopWordsRemover
from pyspark.ml.feature import HashingTF, IDF,Tokenizer, CountVectorizer, CountVectorizer,StringIndexer,StopWordsRemover
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.mllib.classification import SVMModel, SVMWithSGD
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.linalg import Vectors as MLLibVectors
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.ml.feature import * 
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.tuning import ParamGridBuilder


In [3]:
conf = SparkConf().setMaster("local[*]").set("spark.executer.memory", "2g")


In [4]:
sc = SparkContext(conf=conf)
#spark = SparkSession(sc)
spark = SparkSession(sc).builder.getOrCreate()
spark.sparkContext.setLogLevel("ERROR")



Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/28 03:13:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/11/28 03:13:43 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/11/28 03:13:43 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/11/28 03:13:43 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/11/28 03:13:43 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
22/11/28 03:13:43 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.
22/11/28 03:13:43 WARN Utils: Service 'SparkUI' could not bind on port 4045. Attempting port 4046.
22/11/28 03:13:43 WARN Utils: Service 'SparkUI' could not bind on port 4046. Attempting port 4047.
22/11/28 03:13:43 WARN Utils: Service 'SparkUI' could not bind on port 4047. Attempting port 4048.


In [8]:
#"review_id","user_id","business_id","stars","date","text","useful","funny","cool"

#1. Clean the dataset

yelp_rev = spark.read.format("csv").option("quote", "\"").option("escape", "\"").option('multiLine', True).option("encoding", "ISO-8859-1").option("header", "true").load("yelp_review.csv")
yelp_rev = yelp_rev.withColumnRenamed("business_id","business_id_rev")
yelp_rev = yelp_rev.withColumnRenamed("stars","stars_rev")
yelp_rev.printSchema()
yelp_rev.show()



root
 |-- review_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- business_id_rev: string (nullable = true)
 |-- stars_rev: string (nullable = true)
 |-- date: string (nullable = true)
 |-- text: string (nullable = true)
 |-- useful: string (nullable = true)
 |-- funny: string (nullable = true)
 |-- cool: string (nullable = true)

+--------------------+--------------------+--------------------+---------+----------+--------------------+------+-----+----+
|           review_id|             user_id|     business_id_rev|stars_rev|      date|                text|useful|funny|cool|
+--------------------+--------------------+--------------------+---------+----------+--------------------+------+-----+----+
|vkVSCC7xljjrAI4UG...|bv2nCi5Qv5vroFiqK...|AEx2SYEUJmTxVVB18...|        5|2016-05-28|Super simple plac...|     0|    0|   0|
|n6QzIUObkYshz4dz2...|bv2nCi5Qv5vroFiqK...|VR6GpWIda3SfvPC-l...|        5|2016-05-28|Small unassuming ...|     0|    0|   0|
|MV3CcKScW05u5LVfF

In [10]:
yelp_biz = spark.read.format("csv").option("quote", "\"").option("escape", "\"").option('multiLine', True).option("encoding", "ISO-8859-1").option("header", "true").load("yelp_business.csv")
yelp_biz = yelp_biz.withColumnRenamed("business_id","business_id_biz")
yelp_biz = yelp_biz.withColumnRenamed("stars","stars_biz")
yelp_biz.printSchema()


root
 |-- business_id_biz: string (nullable = true)
 |-- name: string (nullable = true)
 |-- neighborhood: string (nullable = true)
 |-- address: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- postal_code: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- stars_biz: string (nullable = true)
 |-- review_count: string (nullable = true)
 |-- is_open: string (nullable = true)
 |-- categories: string (nullable = true)



In [11]:
biz_rev = yelp_rev.join(yelp_biz,yelp_biz.business_id_biz ==  yelp_rev.business_id_rev,"inner")#.show(truncate=False)
biz_rev.show(5)



+--------------------+--------------------+--------------------+---------+----------+--------------------+------+-----+----+--------------------+--------------------+------------+--------------------+---------+-----+-----------+---------+----------+---------+------------+-------+--------------------+
|           review_id|             user_id|     business_id_rev|stars_rev|      date|                text|useful|funny|cool|     business_id_biz|                name|neighborhood|             address|     city|state|postal_code| latitude| longitude|stars_biz|review_count|is_open|          categories|
+--------------------+--------------------+--------------------+---------+----------+--------------------+------+-----+----+--------------------+--------------------+------------+--------------------+---------+-----+-----------+---------+----------+---------+------------+-------+--------------------+
|KEAHrqN-1DV0gmuNH...|xP1IYu2eGfxMWV9tj...|--9e1ONYQuAa-CB_R...|        5|2011-08-24|As part o

In [12]:
biz_rev.createOrReplaceTempView("final_df")


In [15]:
fin = spark.sql("select * from final_df where upper(name) like '%RESTAURANT%'")

fin

DataFrame[review_id: string, user_id: string, business_id_rev: string, stars_rev: string, date: string, text: string, useful: string, funny: string, cool: string, business_id_biz: string, name: string, neighborhood: string, address: string, city: string, state: string, postal_code: string, latitude: string, longitude: string, stars_biz: string, review_count: string, is_open: string, categories: string]

In [14]:
fin = fin.select("review_id","user_id","business_id_rev","stars_rev","text","useful","funny","cool","categories","state","city","categories")


In [16]:
fin1 =  fin.withColumn("stars_rev_double", fin["stars_rev"].cast("double"))


#df3 = df.withColumn("new_gender", expr("CASE WHEN gender = 'M' THEN 'Male' " + "WHEN gender = 'F' THEN 'Female' WHEN gender IS NULL THEN ''" +"ELSE gender END"))


In [17]:
fin1 = fin1.withColumn("label", expr("case when stars_rev_double = 4 then 1 " +"when stars_rev_double = 5 then 1 " +"else 0 end "))
fin1.show(5)

+--------------------+--------------------+--------------------+---------+----------+--------------------+------+-----+----+--------------------+--------------------+------------+--------------------+---------+-----+-----------+-------------+--------------+---------+------------+-------+--------------------+----------------+-----+
|           review_id|             user_id|     business_id_rev|stars_rev|      date|                text|useful|funny|cool|     business_id_biz|                name|neighborhood|             address|     city|state|postal_code|     latitude|     longitude|stars_biz|review_count|is_open|          categories|stars_rev_double|label|
+--------------------+--------------------+--------------------+---------+----------+--------------------+------+-----+----+--------------------+--------------------+------------+--------------------+---------+-----+-----------+-------------+--------------+---------+------------+-------+--------------------+----------------+-----+
|

In [18]:
#"case when stars = 4 then 1 " +"when stars = 5 then 1 " +"else 0 end "
#df = df.withColumn("label", df["stars"].cast("double"))

df = fin1.select('text', 'label')
df.show(3)



+--------------------+-----+
|                text|label|
+--------------------+-----+
|This is currently...|    1|
|This place is awe...|    1|
|Atmosphere for th...|    0|
+--------------------+-----+
only showing top 3 rows



In [19]:
df.createOrReplaceTempView("reviews")
spark.sql("select label,count(label) from reviews group by label").show()



+-----+------------+
|label|count(label)|
+-----+------------+
|    1|      136894|
|    0|       81573|
+-----+------------+



In [20]:
df1 = df.na.drop(how="any")
df1.createOrReplaceTempView("final_df_nonull")



In [21]:
def text_len(text):
    return len(text.split(' '))

len_udf = udf(text_len, IntegerType())



In [22]:
df1 = df1.withColumn('review_len',len_udf('text'))
df1 = df1.filter(df1.review_len > 2)

print("1-- Happy , 0-- Unhappy")
df1.createOrReplaceTempView("df_nonull")
spark.sql("select label,count(label) from df_nonull  group by label").show()



1-- Happy , 0-- Unhappy


+-----+------------+
|label|count(label)|
+-----+------------+
|    1|      136863|
|    0|       81549|
+-----+------------+



In [23]:
import string

def clean_text(string1):
    string1 = string1.lower()
    regex=re.compile('[' + re.escape(string.punctuation) + '0-9\\r\\t\\n]')
    nopunct=regex.sub(" ",string1)
    #pattern=r'[0-9]'
    #text=re.sub(pattern,'',nopunct)
    return nopunct



In [24]:
split_text1=func.udf(lambda x: clean_text(x))
yelp_df=df1.select('text','label',split_text1('text'))


In [26]:
tokenizer = Tokenizer(inputCol="text",outputCol="words")
review_tokenized =tokenizer.transform(yelp_df)
#review_tokenized.show(5)



+--------------------+-----+--------------------+--------------------+
|                text|label|      <lambda>(text)|               words|
+--------------------+-----+--------------------+--------------------+
|This is currently...|    1|this is currently...|[this, is, curren...|
|This place is awe...|    1|this place is awe...|[this, place, is,...|
|Atmosphere for th...|    0|atmosphere for th...|[atmosphere, for,...|
|Great Japanese re...|    1|great japanese re...|[great, japanese,...|
|Cute little hole ...|    1|cute little hole ...|[cute, little, ho...|
+--------------------+-----+--------------------+--------------------+
only showing top 5 rows



In [27]:
stopword_rm = StopWordsRemover(inputCol='words',outputCol='words_new')
review_tokenized = stopword_rm.transform(review_tokenized)


In [29]:
cv=CountVectorizer(inputCol='words_new',outputCol='tfidf')
cvModel=cv.fit(review_tokenized)
count_vectorized=cvModel.transform(review_tokenized)
count_vectorized.show(5)


+--------------------+-----+--------------------+--------------------+--------------------+--------------------+
|                text|label|      <lambda>(text)|               words|           words_new|               tfidf|
+--------------------+-----+--------------------+--------------------+--------------------+--------------------+
|This is currently...|    1|this is currently...|[this, is, curren...|[currently, paren...|(262144,[0,1,18,2...|
|This place is awe...|    1|this place is awe...|[this, place, is,...|[place, awesome!,...|(262144,[0,2,7,15...|
|Atmosphere for th...|    0|atmosphere for th...|[atmosphere, for,...|[atmosphere, rest...|(262144,[0,3,7,9,...|
|Great Japanese re...|    1|great japanese re...|[great, japanese,...|[great, japanese,...|(262144,[0,1,3,6,...|
|Cute little hole ...|    1|cute little hole ...|[cute, little, ho...|[cute, little, ho...|(262144,[0,3,7,12...|
+--------------------+-----+--------------------+--------------------+--------------------+-----

In [30]:
from pyspark.ml.feature import IDF
idf =IDF().setInputCol('tfidf').setOutputCol('tf')
tfidfModel=idf.fit(count_vectorized)
tfidf_df =tfidfModel.transform(count_vectorized)
tfidf_df.show(5)

+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|label|      <lambda>(text)|               words|           words_new|               tfidf|                  tf|
+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
|This is currently...|    1|this is currently...|[this, is, curren...|[currently, paren...|(262144,[0,1,18,2...|(262144,[0,1,18,2...|
|This place is awe...|    1|this place is awe...|[this, place, is,...|[place, awesome!,...|(262144,[0,2,7,15...|(262144,[0,2,7,15...|
|Atmosphere for th...|    0|atmosphere for th...|[atmosphere, for,...|[atmosphere, rest...|(262144,[0,3,7,9,...|(262144,[0,3,7,9,...|
|Great Japanese re...|    1|great japanese re...|[great, japanese,...|[great, japanese,...|(262144,[0,1,3,6,...|(262144,[0,1,3,6,...|
|Cute little hole ...|    1|cute little hole ...|[cute, little

In [31]:
splits = tfidf_df.select(['tfidf', 'label']).randomSplit([0.8,0.2],seed=42)
train = splits[0].cache()
test = splits[1].cache()
train=train.withColumn("label",train.label.cast("int"))
test=test.withColumn("label",test.label.cast("int"))

In [33]:
#df = df.filter(df.label.isin(1.0,5.0))

#df.write.csv("yelp-cleaned")


train_lb = train.rdd.map(lambda row: LabeledPoint(row[1], MLLibVectors.fromML(row[0])))



PythonRDD[184] at RDD at PythonRDD.scala:53

In [34]:
test_lb = test.rdd.map(lambda row: LabeledPoint(row[1], MLLibVectors.fromML(row[0])))


In [35]:
print("----------Logistic Regression------")
from pyspark.ml.classification import LogisticRegression
lr= LogisticRegression(featuresCol ='tfidf',labelCol='label',maxIter=20,regParam=0.3,elasticNetParam=0.8)
lr_model=lr.fit(train)


----------Logistic Regression------


In [36]:
predictions=lr_model.transform(test)
predictions.show()



+--------------------+-----+--------------------+--------------------+----------+
|               tfidf|label|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|(262144,[0,1,2,3,...|    1|[-0.5175591617344...|[0.37342316021035...|       1.0|
|(262144,[0,1,2,3,...|    0|[-0.5175591617344...|[0.37342316021035...|       1.0|
|(262144,[0,1,2,3,...|    1|[-0.5175591617344...|[0.37342316021035...|       1.0|
|(262144,[0,1,2,3,...|    1|[-0.5175591617344...|[0.37342316021035...|       1.0|
|(262144,[0,1,2,3,...|    0|[-0.5175591617344...|[0.37342316021035...|       1.0|
|(262144,[0,1,2,3,...|    1|[-0.5175591617344...|[0.37342316021035...|       1.0|
|(262144,[0,1,2,3,...|    0|[-0.5175591617344...|[0.37342316021035...|       1.0|
|(262144,[0,1,2,3,...|    1|[-0.5175591617344...|[0.37342316021035...|       1.0|
|(262144,[0,1,2,3,...|    0|[-0.5175591617344...|[0.37342316021035...|       1.0|
|(262144,[0,1,2,

In [37]:
acc_eval= MulticlassClassificationEvaluator(labelCol="label",predictionCol="prediction",metricName="accuracy")
acc_eval_lr=acc_eval.evaluate(predictions)
print("Logistic Regression Accuracy :: ",acc_eval_lr)


Logistic Regression Accuracy ::  0.6268299356957177


In [38]:
#RondomForest
print("------Random Forest Accuracy-----")
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(featuresCol = 'tfidf', labelCol = 'label')
rfModel = rf.fit(train)


------Random Forest Accuracy-----


In [39]:
predictions = rfModel.transform(test)
predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|               tfidf|label|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|(262144,[0,1,2,3,...|    1|[7.55842142697927...|[0.37792107134896...|       1.0|
|(262144,[0,1,2,3,...|    0|[8.97593303801791...|[0.44879665190089...|       1.0|
|(262144,[0,1,2,3,...|    1|[8.86269302453433...|[0.44313465122671...|       1.0|
|(262144,[0,1,2,3,...|    1|[7.47283423125702...|[0.37364171156285...|       1.0|
|(262144,[0,1,2,3,...|    0|[7.79448648358644...|[0.38972432417932...|       1.0|
|(262144,[0,1,2,3,...|    1|[7.36709239330246...|[0.36835461966512...|       1.0|
|(262144,[0,1,2,3,...|    0|[8.89527807823118...|[0.44476390391155...|       1.0|
|(262144,[0,1,2,3,...|    1|[7.33164692800473...|[0.36658234640023...|       1.0|
|(262144,[0,1,2,3,...|    0|[9.02387458553125...|[0.45119372927656...|       1.0|
|(262144,[0,1,2,

In [40]:
acc_eval_rf=acc_eval.evaluate(predictions)
print("Random Forest Accuracy :: ",acc_eval_rf)



Random Forest Accuracy ::  0.6273544032471382
